<a href="https://colab.research.google.com/github/olinyoder2534/NLP_practice/blob/main/FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4239657 sha256=8277e6b1f1b476e1054580a97acfcd697957d29bdfac5f1d24a854889967a956
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [2]:
import fasttext

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("/content/Cleaned_Indian_Food_Dataset.csv")

In [5]:
df.head()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12


In [6]:
df.shape

(5938, 9)

In [7]:
df['Cuisine'].value_counts()

Cuisine
Continental             952
Indian                  927
North Indian Recipes    763
South Indian Recipes    558
Italian Recipes         231
                       ... 
Burmese                   1
Shandong                  1
Brunch                    1
Lunch                     1
Dessert                   1
Name: count, Length: 82, dtype: int64

In [8]:
len(pd.unique(df['Cuisine']))

82

In [9]:
import re

In [10]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(r'[ \n]+', ' ', text)
    return text.strip().lower()

In [11]:
df.TranslatedInstructions = df.TranslatedInstructions.map(preprocess)

In [12]:
df.to_csv("food_receipes.txt", columns=["TranslatedInstructions"], header=None, index=False)

In [13]:
model = fasttext.train_unsupervised("food_receipes.txt")

In [14]:
model.get_word_vector("food").shape

(100,)

In [15]:
def get_sentence_vector(text):
    words = text.split()
    vectors = [model.get_word_vector(word) for word in words]
    if vectors:
        sentence_vector = sum(vectors) / len(vectors)
    else:
        sentence_vector = [0] * model.get_dimension()
    return sentence_vector

In [16]:
df['vectors'] = df['TranslatedInstructions'].apply(get_sentence_vector)

In [17]:
df.head()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count,vectors
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,to begin making the masala karela recipe de se...,https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10,"[-0.075723514, 0.07423377, 0.06834873, -0.1147..."
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,to make tomato puliogere first cut the tomatoe...,https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12,"[-0.028595384, 0.13044445, 0.025798932, -0.079..."
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,to begin making the ragi vermicelli recipe fir...,https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12,"[-0.07060176, 0.058530234, 0.051422365, -0.088..."
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,to begin making gongura chicken curry recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15,"[-0.08084306, 0.024845485, 0.021113407, -0.091..."
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,to make andhra style alam pachadi first heat o...,https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12,"[-0.047902934, 0.11059842, 0.029725071, -0.069..."


In [18]:
#remove cusines with less than 100 observations
cuisine_counts = df['Cuisine'].value_counts()
valid_cuisines = cuisine_counts[cuisine_counts >= 100].index
df = df[df['Cuisine'].isin(valid_cuisines)]

In [19]:
df['Cuisine'].value_counts()

Cuisine
Continental              952
Indian                   927
North Indian Recipes     763
South Indian Recipes     558
Italian Recipes          231
Maharashtrian Recipes    152
Bengali Recipes          150
Karnataka                138
Tamil Nadu               133
Kerala Recipes           133
Fusion                   130
Mexican                  116
Andhra                   109
Name: count, dtype: int64

In [20]:
len(pd.unique(df['Cuisine']))

13

In [21]:
df['cuisine_num'] = df['Cuisine'].map({'Continental' : 0, 'Indian': 1, 'North Indian Recipes': 2, 'South Indian Recipes' : 3, 'Italian Recipes': 4, 'Maharashtrian Recipes': 5, 'Bengali Recipes' : 6, 'Karnataka': 7, 'Tamil Nadu': 8,'Kerala Recipes' : 9, 'Fusion': 10, 'Mexican': 11, 'Andhra': 12})
df.head()

<ipython-input-21-a36ef611d94a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cuisine_num'] = df['Cuisine'].map({'Continental' : 0, 'Indian': 1, 'North Indian Recipes': 2, 'South Indian Recipes' : 3, 'Italian Recipes': 4, 'Maharashtrian Recipes': 5, 'Bengali Recipes' : 6, 'Karnataka': 7, 'Tamil Nadu': 8,'Kerala Recipes' : 9, 'Fusion': 10, 'Mexican': 11, 'Andhra': 12})


,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count,vectors,cuisine_num
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,to begin making the masala karela recipe de se...,https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10,"[-0.075723514, 0.07423377, 0.06834873, -0.1147...",1
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,to make tomato puliogere first cut the tomatoe...,https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12,"[-0.028595384, 0.13044445, 0.025798932, -0.079...",3
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,to begin making the ragi vermicelli recipe fir...,https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12,"[-0.07060176, 0.058530234, 0.051422365, -0.088...",3
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,to begin making gongura chicken curry recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15,"[-0.08084306, 0.024845485, 0.021113407, -0.091...",12
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,to make andhra style alam pachadi first heat o...,https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12,"[-0.047902934, 0.11059842, 0.029725071, -0.069...",12


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.vectors.values,
    df.cuisine_num,
    test_size=0.2,
    random_state=42,
    stratify=df.cuisine_num
)

In [23]:
#not going to resample or do any other nlp preprocessing for times sake, would do so in practice

In [24]:
y_train.value_counts()

cuisine_num
0     762
1     742
2     610
3     446
4     185
5     122
6     120
7     110
8     106
9     106
10    104
11     93
12     87
Name: count, dtype: int64

In [25]:
X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf = GradientBoostingClassifier()

clf.fit(X_train_2d, y_train)

y_pred = clf.predict(X_test_2d)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.87      0.80       190
           1       0.40      0.55      0.46       185
           2       0.56      0.62      0.59       153
           3       0.50      0.59      0.54       112
           4       0.76      0.54      0.63        46
           5       0.73      0.27      0.39        30
           6       0.81      0.43      0.57        30
           7       0.50      0.18      0.26        28
           8       0.11      0.04      0.06        27
           9       0.67      0.22      0.33        27
          10       0.00      0.00      0.00        26
          11       0.82      0.39      0.53        23
          12       0.58      0.32      0.41        22

    accuracy                           0.56       899
   macro avg       0.55      0.39      0.43       899
weighted avg       0.56      0.56      0.54       899



In [40]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

clf = Pipeline([
     ('scaler', MinMaxScaler()),
     ('Multi NB', MultinomialNB())
])
clf.fit(X_train_2d, y_train)

y_pred = clf.predict(X_test_2d)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.86      0.68       190
           1       0.29      0.62      0.39       185
           2       0.52      0.42      0.47       153
           3       0.52      0.40      0.45       112
           4       0.00      0.00      0.00        46
           5       0.00      0.00      0.00        30
           6       0.00      0.00      0.00        30
           7       0.00      0.00      0.00        28
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        27
          10       0.00      0.00      0.00        26
          11       0.00      0.00      0.00        23
          12       0.00      0.00      0.00        22

    accuracy                           0.43       899
   macro avg       0.15      0.18      0.15       899
weighted avg       0.33      0.43      0.36       899



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
df1 = pd.read_csv('/content/Ecommerce_data.csv')
df1.head()

,Text,label
0,Urban Ladder Eisner Low Back Study-Office Comp...,Household
1,"Contrast living Wooden Decorative Box,Painted ...",Household
2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics
3,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing & Accessories
4,Indira Designer Women's Art Mysore Silk Saree ...,Clothing & Accessories


In [31]:
df1.shape

(24000, 2)

In [34]:
df1['label'].value_counts()

label
Household                 6000
Electronics               6000
Clothing & Accessories    6000
Books                     6000
Name: count, dtype: int64

In [36]:
df1.dropna(inplace = True)
df1.shape

(24000, 2)

In [42]:
df1.label.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

In [47]:
df1.label.unique()

array(['Household', 'Electronics', 'Clothing_Accessories', 'Books'],
      dtype=object)

In [49]:
df1['label_new'] = '__label__' + df1['label'].astype(str)
df1.head()

,Text,label,label_new
0,Urban Ladder Eisner Low Back Study-Office Comp...,Household,__label__Household
1,"Contrast living Wooden Decorative Box,Painted ...",Household,__label__Household
2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics,__label__Electronics
3,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing_Accessories,__label__Clothing_Accessories
4,Indira Designer Women's Art Mysore Silk Saree ...,Clothing_Accessories,__label__Clothing_Accessories


In [51]:
df1['textLabel'] = df1['label_new'] + ' ' + df1['Text']
df1.head()

,Text,label,label_new,textLabel
0,Urban Ladder Eisner Low Back Study-Office Comp...,Household,__label__Household,__label__Household Urban Ladder Eisner Low Bac...
1,"Contrast living Wooden Decorative Box,Painted ...",Household,__label__Household,__label__Household Contrast living Wooden Deco...
2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics,__label__Electronics,__label__Electronics IO Crest SY-PCI40010 PCI ...
3,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing_Accessories,__label__Clothing_Accessories,__label__Clothing_Accessories ISAKAA Baby Sock...
4,Indira Designer Women's Art Mysore Silk Saree ...,Clothing_Accessories,__label__Clothing_Accessories,__label__Clothing_Accessories Indira Designer ...


In [52]:
import re

def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [53]:
df1['textLabel'] = df1['textLabel'].map(preprocess)
df1.head()

,Text,label,label_new,textLabel
0,Urban Ladder Eisner Low Back Study-Office Comp...,Household,__label__Household,__label__household urban ladder eisner low bac...
1,"Contrast living Wooden Decorative Box,Painted ...",Household,__label__Household,__label__household contrast living wooden deco...
2,IO Crest SY-PCI40010 PCI RAID Host Controller ...,Electronics,__label__Electronics,__label__electronics io crest sy pci40010 pci ...
3,ISAKAA Baby Socks from Just Born to 8 Years- P...,Clothing_Accessories,__label__Clothing_Accessories,__label__clothing_accessories isakaa baby sock...
4,Indira Designer Women's Art Mysore Silk Saree ...,Clothing_Accessories,__label__Clothing_Accessories,__label__clothing_accessories indira designer ...


In [55]:
train, test = train_test_split(df1, test_size=0.2)

In [56]:
train.to_csv("ecommerce.train", columns=["textLabel"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["textLabel"], index=False, header=False)

In [57]:
model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

(4800, 0.9720833333333333, 0.9720833333333333)

In [ ]:
#97% precision & recall

In [60]:
model

In [69]:
model.predict("Sweatshit with a hoodie and good for temperatures over 30 degrees F")
#incorrect prediction

(('__label__books',), array([0.80897659]))

In [59]:
model.predict("Designer shirt made with silk and satin")

(('__label__clothing_accessories',), array([0.99954474]))

In [62]:
model.predict("desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99905139]))

In [63]:
model.predict("large couch with room for 5 guests")

(('__label__household',), array([0.99896657]))